### 处理our_dateset(合并通道,颜色反转,归一化)

In [ ]:

import os
import cv2

src_path = "dataset/CUHK/DATA_512/Test/gt"
tar_path = "Results/AUPSNet_gt"
for file in os.listdir(src_path):
    src = os.path.join(src_path,file)
    tar = os.path.join(tar_path,file)
    img = cv2.imread(src)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    cv2.imwrite(tar,img)

### 绘制PR曲线

In [ ]:
from PIL import Image
import os
import numpy as np
import shutil
import os
import cv2 

### 计算召回率和准确率

In [ ]:
def PrLine(GT,result,thread):
    
    
    GT_img = Image.open(GT)
    W,H = GT_img.size
    if H < W and W != 640:
        GT_img = GT_img.resize((640,H))
    
    if H > W and H != 640:
        GT_img = GT_img.resize((W,640))
    
    
    result_img = Image.open(result)
    result_img = result_img.resize((W,H))
    kW,kH = result_img.size
    if kH < kW and kW != 640:
        result_img = result_img.resize((640,kH))
    
    if kH > kW and kH != 640:
        Gresult_img = result_img.resize((kW,640))
    
    
    GT_img = np.asarray(GT_img)
    result_img = np.asarray(result_img)
    
    
    GT_img = GT_img.astype(np.float)
    result_img = result_img.astype(np.float)
    #print(result_img)
    
    GT_img = (GT_img>0.5)
    result_img = (result_img>thread) #32 64 96 128 160 192 224 254
    
    
    U_img = GT_img*result_img
    GT_num = GT_img.sum()
    
    
    result_num = result_img.sum()
    U_num = U_img.sum()
    if result_num == 0:
        result_num = 0.0000001
    
    if GT_num == 0:
        GT_num = 0.0000001
    
    Precision = U_num/result_num
    
    Recall = U_num/GT_num
    
    
    
    return (Precision,Recall)


def PRlist(GT_path,output_path):
    num_list = os.listdir(GT_path)

    P = []
    R = []
    
    for thread in range(255):
        
        Precision_list = []
        Recall_list = []
        for p in num_list:
            GT_img = os.path.join(GT_path, p) 
            result_img = os.path.join(output_path, p[:-4] + ".png")
            Pr,Re = PrLine(GT_img,result_img,thread)
            Precision_list.append(Pr)
            Recall_list.append(Re)
            
        P.append(np.mean(Precision_list))
        R.append(np.mean(Recall_list))

    return (P,R)

### PR曲线绘制函数

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
#from scipy.interpolate import curve_fit


def drawPR(path,P_all,R_all,colors):
    #colors = ['red','green','yellow','antiquewhite','darkorange','aqua','aquamarine','darkgreen','beige','bisque','black','blanchedalmond','blue','brown','burlywood','cadetblue','chartreuse','chocolate','coral'] #,'blueviolet'
    for i in range(len(path)):
        r_p = np.array(R_all[i]) 
        p_p = np.array(P_all[i])
        r_p = r_p[:-1]
        p_p = p_p[:-1]
        plt.plot(r_p,p_p,label=path[i],color=colors[i])
        

    plt.xlabel('Recall')
    plt.ylabel('Precison')
    plt.legend(loc='lower right')
    plt.savefig("PR Curve.png");
    plt.show()

In [ ]:
GT_path = "Results/GT/"
output_path = "Results/"

#path = []
#path = ['DC','Patch_DC','LinkNet_new','Unet_crf','Unet','MIUnet','DeFusionNet',"AUSPNet",'Chakrabarti','DBDF','DHDE','HiFST','JNB','KSFV','LBP','SVD','SS']
path = ['DeFusionNet','DBDF','HiFST','JNB','LBP','SS']
P_all = []
R_all = []

for pp in path:
    Out_p = output_path + pp + '/'
    
    p,r = PRlist(GT_path,Out_p)
    P_all.append(p)
    R_all.append(r)

#print(P_all)result_img,
#print(R_all)

In [ ]:
ourgt_path = "Results/AUPSNet_gt/"
output_path = "Results/AUPSNet_results/"

p,r = PRlist(ourgt_path,output_path)
P_all.append(p)
R_all.append(r)
path.append("AUPSNet")

In [ ]:
ourgt_path = "Results/BTBNet_gt/"
output_path = "Results/BTBNet_results/"

p,r = PRlist(ourgt_path,output_path)
P_all.append(p)
R_all.append(r)
path.append("BTBNet")

In [ ]:
ourgt_path = "Results/CENet_gt/"
output_path = "Results/CENet_results/"

p,r = PRlist(ourgt_path,output_path)
P_all.append(p)
R_all.append(r)
path.append("CENet")

In [ ]:
colors = ['red','green','yellow','darkorange','aqua','darkgreen','blue','brown','burlywood','cadetblue','chartreuse','chocolate','coral'] #,'blueviolet'
drawPR(path,P_all,R_all, colors)
print(len(P_all))

###挑选训练集中的gt

In [ ]:
'''
import shutil
GT_path = "/home/ailab/data1/gwl/模糊检测/Unet/data/tt/data/DUT_GT/"
Out_p = "/home/ailab/data1/gwl/U-2-Net/test_data/DUT_test_source/"
copy_path = "/home/ailab/data1/gwl/U-2-Net/test_data/DUT_test_gt/"

for file in os.listdir(Out_p):
    tar = os.path.join(GT_path,file[:-3]+"png")
    #img = cv2.imread(tar)
    #img = 255 - cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #cv2.imwrite(os.path.join(copy_path,file),img)
    shutil.copyfile(tar,os.path.join(copy_path,file[:-3]+"png"))
    
